In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import copy

class item:
    def __init__(self, item_id, item_weight):
        self.item_id = item_id
        self.item_weight = item_weight
        
class bin:
    def __init__(self, bin_id, bin_fitness):
        self.bin_id = bin_id
        self.bin_fitness = bin_fitness
        self.items_in_bin = []

class individual:
    def __init__(self, individual_id, individual_fitness, bins_in_individual):
        self.individual_id = individual_id
        self.individual_fitness = individual_fitness
        self.bins_in_individual = bins_in_individual

In [30]:
#generates items
def generateItems(num_items_to_generate, bin_capacity):
    items = []
    for i in range(num_items_to_generate):
        items.append(item(i, round(random.uniform(0.0, bin_capacity), 2))) #self.item_id = i; self.item_weight = random.uniform(0.0, bin_capacity); self.belongs_to_bin = None;
    return items

#places Items on bin
def placeItemsOnBin(items, bin_capacity):
    empty_bins = []
    empty_bins.append(bin(0, 0)) #create very first bin
    #first come first serve (put item in bin from array of items as it comes and create new bin if a bin overflows)
    item_index = 0
    i = 0
    while i < len(empty_bins):
        for j in range(item_index, len(items)):
            if empty_bins[i].bin_fitness + items[j].item_weight <  bin_capacity: #if adding next item wont overflow the bin
                empty_bins[i].items_in_bin.append(items[j]) #add item_id to array items_in_bin of the bin
                empty_bins[i].bin_fitness += items[j].item_weight #update fitness
                item_index += 1
            elif empty_bins[i].bin_fitness + items[j].item_weight >= bin_capacity:
                empty_bins.append(bin(i+1, 0))
                i += 1
                break
        if item_index == len(items):   #terminates the loop
            i += 1    
    return empty_bins

#calculates individuals fitness in a population
def calculate_fitness(bins, k, bin_capacity):
    n = len(bins)
    sum_fitness = 0
    for b in bins:
        sum_fitness += (b.bin_fitness / bin_capacity) ** k
    sum_fitness = sum_fitness / n
    return sum_fitness

#tournament selection
def tournamentSelection(individuals, tournament_size):
    selected = []
    while len(selected) < 2:
        best_individual = []
        tournament = random.sample(individuals, tournament_size)
        best_individual.append(max(tournament, key=lambda r: r.individual_fitness))
        selected.append(best_individual[0].bins_in_individual)
    return selected

# Perform Crosover
def crossover(parent1, parent2, bin_capacity, items):
    #clone/copy the parents
    parent1 = copy.deepcopy(parent1)
    parent2 = copy.deepcopy(parent2)
    
    #pick two crossover points
    p1_point1 = random.sample(range(len(parent1)), 1)
    p1_point2 = p1_point1
    while p1_point1 == p1_point2:
        p1_point2 = random.sample(range(len(parent1)), 1)
    
    #have values in sorted order
    p1_point1, p1_point2 = sorted([p1_point1, p1_point2])
    p1_point1, p1_point2 = p1_point1[0], p1_point2[0]
    
    #print(p1_point1, p1_point2)

    p2_point1 = random.sample(range(len(parent2)), 1)
    p2_point2 = p2_point1
    while p2_point1 == p2_point2:
        p2_point2 = random.sample(range(len(parent2)), 1)
    
    #print('finished picking crossover point')
    
    #have values in sorted order
    p2_point1, p2_point2 = sorted([p2_point1, p2_point2])
    p2_point1, p2_point2 = p2_point1[0], p2_point2[0]
    
    #print(p2_point1, p2_point2)

    #Temporarily save the selected chromosome/bins of each parent
    chromosome_to_parent1 = parent2[p2_point1:p2_point2] #chromosome that will go from parent 1 to parent 2
    chromosome_to_parent2 = parent1[p1_point1:p1_point2] #chromosome that will go from parent 2 to parent 1

    #print('begin Find duplicate items in selected bins')
    
    #find duplicate items in selected bins from parents 1 and parents 2
    #array of selected item id's to use built in python function
    parent1_to_parent2_item_id = [] #holds item's id that will go to parent 1 from parent 2
    parent2_to_parent1_item_id = [] #holds selected item's ids that will go to parent 2 from parent 1
    for chromosome in chromosome_to_parent1:
        for gene in chromosome.items_in_bin:
            parent1_to_parent2_item_id.append(gene.item_id)
            
    for chromosome in chromosome_to_parent2:
        for gene in chromosome.items_in_bin:
            parent2_to_parent1_item_id.append(gene.item_id)
    #print('End')
    
    #find and insert non duplicate id's
    left_over_item_id_for_parent1 = [] #stores leftover items to be placed in parent1
    left_over_item_id_for_parent2 = [] #store leftover items to be placed in parent2
    
    #for outgoing items from parent 2, if items not in bin, add as leftover item
    for i in parent2_to_parent1_item_id:
        if i not in parent1_to_parent2_item_id:
            left_over_item_id_for_parent1.append(i)
    
    for i in parent1_to_parent2_item_id:
        if i not in parent2_to_parent1_item_id:
            left_over_item_id_for_parent2.append(i)
    
    #print('bins_to_parent1:', chromosome_to_parent1)
    #print('bins_to_parent2:', chromosome_to_parent2)

    #------------------------------------------
    #print(len(parent1), len(parent2))
    
    #print('remove selected bins from parents')
    #remove selected bins from the parent1
    parent1[p1_point1:p1_point2] = []
    #removing duplicate bins and finding other leftover item id's for parent 1
    for chromosome in parent1:
        items_in_bin_by_id = []
        is_duplicate = False
        for gene in chromosome.items_in_bin:
            items_in_bin_by_id.append(gene.item_id)
        for i in parent2_to_parent1_item_id:
            if i in items_in_bin_by_id:
                is_duplicate = True
                break
        if is_duplicate == True:
            #append non duplicate item's item_id to left_over item_id's list
            for i in parent2_to_parent1_item_id:
                if i not in items_in_bin_by_id:
                    left_over_item_id_for_parent1.append(i)
            #remove bin from individual
            parent1.remove(chromosome)
    
    #print(len(parent1), len(parent2))
    #-------------------------------------------------------------
    #remove selected bins from the parent2
    parent2[p2_point1:p2_point2] = []
    #removing duplicate bins and finding other leftover item id's for parent 2
    for chromosome in parent2:
        items_in_bin_by_id = []
        is_duplicate = False
        for gene in chromosome.items_in_bin:
            items_in_bin_by_id.append(gene.item_id)
        for i in parent1_to_parent2_item_id:
            if i in items_in_bin_by_id:
                is_duplicate == True
                break
        if is_duplicate == True:
            #append non duplicate item's item_id to left_over item_id's list
            for i in parent1_to_parent2_item_id:
                if i not in items_in_bin_by_id:
                    left_over_item_id_for_parent2.append(i)
            #remove bin from individual
            parent2.remove(chromosome)
            
    #print(len(parent1), len(parent2))
   #-------------------------------------------------------------         
    #print('after removing bins from parent1:', parent1)
    #print('after removing bins from parent2:', parent2)

    #place/swap selected chromosome between parents (remove selected bins from the parent1 and place it to another parent and vice versa)
    parent1[p1_point1:p1_point1] = chromosome_to_parent1
    parent2[p2_point1:p2_point1] = chromosome_to_parent2
    
    print(len(parent1), len(parent2))
    
    #print('Inserting unassigned items back into solution')
    #Insert unassigned items back into solution
    #sort leftover items in descending order and place it into first bin that has enough space
    # replace item_ids with item object (items that should go into parent 1)
    left_over_item_object_for_parent1 = []
    left_over_item_object_for_parent2 = []
    
    for i in left_over_item_id_for_parent1:
        for j in items:
            if i == j.item_id:
                left_over_item_object_for_parent1.append(j)
        
    #replace item_ids with item object (item that should go into parent 2)
    for i in left_over_item_id_for_parent2:
        for j in items:
            if i == j.item_id:
                left_over_item_object_for_parent2.append(j)
                
    #print('sorting remaining items with no bins')
    #sort remaining items in descending order (by item weight)
    sorted(left_over_item_object_for_parent1, key=lambda x: x.item_weight, reverse=True)
    sorted(left_over_item_object_for_parent2, key=lambda x: x.item_weight, reverse=True)
    
    #print('placing items into bins that fit')
    #print('apple', len(left_over_item_object_for_parent1), len(left_over_item_id_for_parent1))
    #place sorted item into bin that fits else create a new bin (for parent1)
    for gene in left_over_item_object_for_parent1:
        found_fit = False
        for chromosome in parent1:
            if (chromosome.bin_fitness + gene.item_weight) <= bin_capacity:
                chromosome.items_in_bin.append(gene)
                chromosome.bin_fitness += gene.item_weight
                found_fit = True
                break
        if found_fit == False:
            parent1.append(bin(len(parent1) + 1, gene.item_weight))
            parent1[-1].items_in_bin.append(gene)
            
    #print('ball', len(left_over_item_object_for_parent2), len(left_over_item_id_for_parent2))
    #place sorted item into bin that fits else create a new bin (for parent2)

    for gene in left_over_item_object_for_parent2:
        found_fit = False
        for chromosome in parent2:
            if (chromosome.bin_fitness + gene.item_weight) <= bin_capacity:
                chromosome.items_in_bin.append(gene)
                chromosome.bin_fitness += gene.item_weight
                found_fit = True
                break
        if found_fit == False:
            parent2.append(bin(len(parent1) + 1, gene.item_weight))
            parent2[-1].items_in_bin.append(gene)
    #print('End')
    #reassign bin_id
    index = 0
    for i in parent1:
        i.bin_id = index
        index += 1
    index = 0
    for i in parent2:
        i.bin_id = index
        index += 1
    
    children = [parent1, parent2]
    return children

#destroy bins randomly, rearrange the items from destroyed bin in descending order, place items into remaining bins if it fits
#else create a new bin
def mutation(child, bin_capacity):
    num_items_to_destroy = random.randrange(0, len(child))
    remaining_items_after_bins_destroyed = []
    
    #removing bins
    for i in range(num_items_to_destroy):
        index = random.randrange(0, len(child))
        for items in child[index].items_in_bin: #child[i] = bin
            remaining_items_after_bins_destroyed.append(items)
        child.pop(index)
    
    #sort and reinsert items to the bin
    sorted(remaining_items_after_bins_destroyed, key=lambda x: x.item_weight, reverse=True)
    
    #reinsert
    for gene in remaining_items_after_bins_destroyed: #gene = item
        is_fit = False
        for chromosome in child:#chromosome = bin
            if (chromosome.bin_fitness + gene.item_weight) <= bin_capacity:
                chromosome.items_in_bin.append(gene)
                chromosome.bin_fitness += gene.item_weight
                is_fit = True
                break
        if is_fit == True:
            child.append(bin(0, gene.item_weight))
            child[-1].items_in_bin.append(gene)
    #reassign bin_id
    index = 0
    for i in child:
        i.bin_id = index
        index += 1
    
    return child
    

In [31]:
bin_capacity = 10.0
num_items_to_generate = 100
items = generateItems(num_items_to_generate, bin_capacity)
generation_to_generate = 3
current_generation = 0
population_size = 100
tournament_size = 100
least_bins_per_gen = []
best_5_individuals_per_gen_with_best_fitness = []

#initialize individuals
individuals = []
for i in range(population_size):
    random.shuffle(items)
    individuals.append(placeItemsOnBin(items, bin_capacity))

#initializing population by adding individuals in it (individuals fitness calculated here)
population = []
for i in range(len(individuals)):
    population.append(individual(i, calculate_fitness(individuals[i], 2, bin_capacity), individuals[i]))

#individual with least amount of bins/best fitness for initial population
individual_with_max_fitness = max(population, key=lambda x: x.individual_fitness)
least_bins_per_gen.append(len(individual_with_max_fitness.bins_in_individual))

current_population = population
while current_generation < generation_to_generate:
    children = []
    while len(children) < population_size:
        # selecting parents via tournament selection
        parents = tournamentSelection(population, tournament_size)
        parent1 = parents[0]
        parent2 = parents[1]
        
        #Apply Crossover and mutation method
        child = crossover(parent1, parent2, bin_capacity, items)
        
        #5% change of mutation
        if random.randrange(0, 20) == 5:
            child[0] = mutation(child[0], bin_capacity)
            
        #print(len(child[0]))
        #print(len(child[1]))
        children.append(child[0]) #children = array of bins
        children.append(child[1])
            
    #calculate fitness of all children
    print('complete gen', current_generation)
    current_population = []
    for i in range(len(children)):
        current_population.append(individual(i, calculate_fitness(children[i], 2, bin_capacity), children[i]))
    child_with_max_fitness = max(current_population, key=lambda x: x.individual_fitness)
    least_bins_per_gen.append(len(child_with_max_fitness.bins_in_individual))
    current_generation += 1

28 94
68 54
22 100
56 66
33 89
72 50
65 57
74 48
45 77
101 21
55 67
60 62
57 65
75 47
78 44
47 75
39 83
23 99
56 66
77 45
90 32
21 101
39 83
49 73
68 54
72 50
85 37
51 71
42 80
109 13
37 85
69 53
44 78
32 90
64 58
28 94
51 71
58 64
98 24
83 39
41 81
76 46
56 66
61 61
50 72
105 17
47 75
80 42
46 76
61 61
complete gen 0
75 47
77 45
43 79
87 35
12 110
41 81
84 38
32 90
93 29
91 31
40 82
83 39
69 53
81 41
97 25
94 28
47 75
63 59
63 59
60 62
61 61
53 69
71 51
48 74
33 89
65 57
46 76
92 30
76 46
53 69
56 66
71 51
39 83
79 43
34 88
105 17
65 57
64 58
51 71
79 43
65 57
41 81
54 68
41 81
87 35
68 54
74 48
76 46
47 75
55 67
complete gen 1
91 31
44 78
45 77
55 67
50 72
77 45
33 89
94 28
30 92
49 73
68 54
65 57
49 73
17 105
69 53
64 58
99 23
95 27
41 81
98 24
27 95
35 87
72 50
76 46
94 28
64 58
98 24
35 87
52 70
62 60
33 89
49 73
56 66
40 82
53 69
70 52
40 82
88 34
42 80
50 72
55 67
93 29
47 75
51 71
65 57
24 98
56 66
81 41
41 81
102 20
complete gen 2


In [15]:
least_bins_per_gen

[64, 54, 60, 107]

Testing crossover function

In [54]:
parent1 = [1,2,3,4,5]
parent2 = [5,4,3,2,1]
    
p1_point1 = random.sample(range(len(parent1)), 1)
p1_point2 = p1_point1
while p1_point1 == p1_point2:
    p1_point2 = random.sample(range(len(parent1)), 1)

#have values in sorted order
p1_point1, p1_point2 = sorted([p1_point1, p1_point2])
p1_point1, p1_point2 = p1_point1[0], p1_point2[0]

print(p1_point1, p1_point2)

p2_point1 = random.sample(range(len(parent2)), 1)
p2_point2 = p2_point1
while p2_point1 == p2_point2:
    p2_point2 = random.sample(range(len(parent2)), 1)

#have values in sorted order
p2_point1, p2_point2 = sorted([p2_point1, p2_point2])
p2_point1, p2_point2 = p2_point1[0], p2_point2[0]
        
print(p2_point1, p2_point2)

#Temporarily save the selected chromosome of each parent
chromosome_to_parent1 = parent2[p2_point1:p2_point2]
chromosome_to_parent2 = parent1[p1_point1:p1_point2]

print('chromosome_to_parent1:', chromosome_to_parent1)
print('chromosome_to_parent2:', chromosome_to_parent2)

#remove selected chromosome from the parents
parent1[p1_point1:p1_point2] = []
parent2[p2_point1:p2_point2] = []

print('after removing chromosome from parent1:', parent1)
print('after removing chromosome from parent2:', parent2)

#place/swap selected chromosome between parents
parent1[p1_point1:p1_point1] = chromosome_to_parent1
parent2[p2_point1:p2_point1] = chromosome_to_parent2

parent1

1 2
1 3
chromosome_to_parent1: [4, 3]
chromosome_to_parent2: [2]
after removing chromosome from parent1: [1, 3, 4, 5]
after removing chromosome from parent2: [5, 2, 1]


[1, 4, 3, 3, 4, 5]

In [55]:
parent2

[5, 2, 2, 1]

In [48]:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Define the start and end indices of the sequence you want to remove
start_index = 2
end_index = 5

# Remove the sequence using slicing
my_list[start_index:end_index+1] = []

print(my_list)

[1, 2, 7, 8, 9]


In [166]:
a = random.sample(range(len((parent1))), 1)
a

[3]

In [58]:
apple = [bin(0,0), bin(1,0), bin(2,0)]
ball = apple[1]
apple.remove(ball)

In [60]:
for i in apple:
    print(i.bin_id)

0
2


1